In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [2]:
splits = ['train[:60%]', 'train[-40%:]', 'test']

splits, info = tfds.load(
    name="imdb_reviews",
    with_info=True,
    split=splits,
    as_supervised=True
)
train_data, validation_data, test_data = splits

Shuffling and writing examples to /Users/shankar/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete49K60P/imdb_reviews-train.tfrecord
Shuffling and writing examples to /Users/shankar/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete49K60P/imdb_reviews-test.tfrecord
Shuffling and writing examples to /Users/shankar/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete49K60P/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /Users/shankar/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### Explore the Data

In [3]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
num_classes = info.features['label'].num_classes

print('The dataset has a total of:')
print(f"{num_classes} classes")
print(f"{num_train_examples} movie reviews for training")
print(f"{num_test_examples} movie reviews for testing")

The dataset has a total of:
2 classes
25000 movie reviews for training
25000 movie reviews for testing


In [4]:
class_names = ['negative', 'positive']

In [5]:
for review, label in train_data.take(1):
    review = review.numpy()
    label = label.numpy()
    
    print(f"Movie Review: {review}")
    print(f"Label: {class_names[label]}")

Movie Review: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label: negative


### Load Word Embeddings

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

### Build Pipeline

In [7]:
batch_size = 512

train_batches = train_data.shuffle(num_train_examples // 4).batch(batch_size).prefetch(1)
validation_batches = validation_data.batch(batch_size).prefetch(1)
test_batches = test_data.batch(batch_size)

### Build the Model

In [8]:
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_batches,
    epochs=20,
    validation_data=validation_batches
)

Epoch 1/20
30/30 [==============================] - 3s 89ms/step - loss: 0.7922 - accuracy: 0.5173 - val_loss: 0.7088 - val_accuracy: 0.5608
Epoch 2/20
30/30 [==============================] - 2s 70ms/step - loss: 0.6616 - accuracy: 0.6093 - val_loss: 0.6336 - val_accuracy: 0.6477
Epoch 3/20
30/30 [==============================] - 2s 72ms/step - loss: 0.6063 - accuracy: 0.6791 - val_loss: 0.5895 - val_accuracy: 0.6912
Epoch 4/20
30/30 [==============================] - 2s 74ms/step - loss: 0.5531 - accuracy: 0.7259 - val_loss: 0.5367 - val_accuracy: 0.7408
Epoch 5/20
30/30 [==============================] - 2s 71ms/step - loss: 0.4998 - accuracy: 0.7740 - val_loss: 0.4906 - val_accuracy: 0.7760
Epoch 6/20
30/30 [==============================] - 2s 74ms/step - loss: 0.4527 - accuracy: 0.8031 - val_loss: 0.4554 - val_accuracy: 0.7977
Epoch 7/20
30/30 [==============================] - 2s 72ms/step - loss: 0.4123 - accuracy: 0.8265 - val_loss: 0.4244 - val_accuracy: 0.8140
Epoch 8/20
30

In [11]:
eval_results = model.evaluate(test_batches, verbose=0)
for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.3}'.format(value))

loss: 0.324
accuracy: 0.862
